In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import pickle
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
import orjson

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
with open('dataset_test_files.pkl', 'rb') as fp:
    df_test = pickle.load(fp)
df_test.head()

,filename,kind,defect,size,class_name,class
3,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
4,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
11,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
15,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
20,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [3]:
with open('dataset_train_files.pkl', 'rb') as fp:
    df_train = pickle.load(fp)
df_train.head()

,filename,kind,defect,size,class_name,class
0,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
1,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
2,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
5,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
6,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [4]:
gen_train = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # zoom_range=[1, 3],
    horizontal_flip=True,
    vertical_flip=True
)
gen_train

In [5]:
flow_test  = gen_train.flow_from_dataframe(df_test, y_col='class_name', batch_size=32)
flow_train  = gen_train.flow_from_dataframe(df_train, y_col='class_name', batch_size=32)
flow_test, flow_train

Found 360 validated image filenames belonging to 4 classes.
Found 840 validated image filenames belonging to 4 classes.


(<keras.preprocessing.image.DataFrameIterator at 0x23b85bcbec8>,
 <keras.preprocessing.image.DataFrameIterator at 0x23bc0250d88>)

In [6]:
dir(flow_test)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_params',
 '_filepaths',
 '_filter_classes',
 '_filter_valid_filepaths',
 '_flow_index',
 '_get_batches_of_transformed_samples',
 '_keras_api_names',
 '_keras_api_names_v1',
 '_sample_weight',
 '_set_index_array',
 'allowed_class_modes',
 'batch_index',
 'batch_size',
 'class_indices',
 'class_mode',
 'classes',
 'color_mode',
 'data_format',
 'directory',
 'dtype',
 'filenames',
 'filepaths',
 'get_classes',
 'image_data_generator',
 'image_shape',
 'index_array',
 'index_generator',
 'interpolation',
 'labels',
 'lock',
 'n',
 'next',
 'on_epoch_end',
 'reset'

In [6]:
df_test[df_test['class'] != flow_test.classes]

,filename,kind,defect,size,class_name,class


In [7]:
df_train[df_train['class'] != flow_train.classes]

,filename,kind,defect,size,class_name,class


In [8]:
model = keras.Sequential()

model.add(Conv2D(16, (5, 5), activation='relu', input_shape=(256, 256, 3))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [9]:
model.compile(optimizer='adam', # Rotina de otimização, que informa a ml como ajustar o valor dos parâmetros para minimizar o erro.
              loss='categorical_crossentropy', # Função de erro que diz o quão erradas estão nossas predições
              metrics=['accuracy']) # Lista de métricas para avaliar o nosso modelo

In [10]:
with open('model_da_dropout.json', 'w') as fp:
    fp.write(model.to_json())

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 252, 252, 16)      1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 16)       

In [12]:
history = model.fit_generator(flow_train, epochs=100, verbose=2, validation_data=flow_test, callbacks=[
    ModelCheckpoint(
        'model_da_dropout.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs_da_dropout')
])

Epoch 1/100
 - 10s - loss: 2.4759 - accuracy: 0.2464 - val_loss: 1.3672 - val_accuracy: 0.2472
Epoch 2/100
 - 9s - loss: 1.3892 - accuracy: 0.2345 - val_loss: 1.3846 - val_accuracy: 0.3333
Epoch 3/100
 - 9s - loss: 1.3876 - accuracy: 0.2512 - val_loss: 1.4008 - val_accuracy: 0.3306
Epoch 4/100
 - 9s - loss: 1.3845 - accuracy: 0.2988 - val_loss: 1.3880 - val_accuracy: 0.3194
Epoch 5/100
 - 9s - loss: 1.3855 - accuracy: 0.2964 - val_loss: 1.3755 - val_accuracy: 0.3139
Epoch 6/100
 - 9s - loss: 1.3839 - accuracy: 0.3000 - val_loss: 1.3771 - val_accuracy: 0.3056
Epoch 7/100
 - 8s - loss: 1.3850 - accuracy: 0.2702 - val_loss: 1.3182 - val_accuracy: 0.2861
Epoch 8/100
 - 9s - loss: 1.3857 - accuracy: 0.2857 - val_loss: 1.3454 - val_accuracy: 0.2639
Epoch 9/100
 - 9s - loss: 1.3812 - accuracy: 0.3036 - val_loss: 1.3568 - val_accuracy: 0.2750
Epoch 10/100
 - 9s - loss: 1.3846 - accuracy: 0.2774 - val_loss: 1.3720 - val_accuracy: 0.3056
Epoch 11/100
 - 9s - loss: 1.3815 - accuracy: 0.2774 - val

In [13]:
with open('train_da_history.json', 'wb') as fp:
    fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history.history.items()}))